# Welcome! You are now in DataLab.
You successfully completed your project and are looking for some additional related challenges. This DataLab workbook contains the official solution from our curriculum staff, along with Additional Challenges at the bottom. If you would like a quick overview of DataLab, please refer to the help menu. You can easily share your project with your friends and colleagues when you're done.

Good luck with your additional challenges!

![Parked motorcycle](motorcycle.jpg)

You're working for a company that sells motorcycle parts, and they've asked for some help in analyzing their sales data!

They operate three warehouses in the area, selling both retail and wholesale. They offer a variety of parts and accept credit cards, cash, and bank transfer as payment methods. However, each payment type incurs a different fee.

The board of directors wants to gain a better understanding of wholesale revenue by product line, and how this varies month-to-month and across warehouses. You have been tasked with calculating net revenue for each product line and grouping results by month and warehouse. The results should be filtered so that only `"Wholesale"` orders are included.

They have provided you with access to their database, which contains the following table called `sales`:

## Sales
| Column | Data type | Description |
|--------|-----------|-------------|
| `order_number` | `VARCHAR` | Unique order number. |
| `date` | `DATE` | Date of the order, from June to August 2021. |
| `warehouse` | `VARCHAR` | The warehouse that the order was made from&mdash; `North`, `Central`, or `West`. |
| `client_type` | `VARCHAR` | Whether the order was `Retail` or `Wholesale`. |
| `product_line` | `VARCHAR` | Type of product ordered. |
| `quantity` | `INT` | Number of products ordered. | 
| `unit_price` | `FLOAT` | Price per product (dollars). |
| `total` | `FLOAT` | Total price of the order (dollars). |
| `payment` | `VARCHAR` | Payment method&mdash;`Credit card`, `Transfer`, or `Cash`. |
| `payment_fee` | `FLOAT` | Percentage of `total` charged as a result of the `payment` method. |


Your query output should be presented in the following format:

| `product_line` | `month` | `warehouse` |	`net_revenue` |
|----------------|-----------|----------------------------|--------------|
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_two | --- | --- | --- |
| ... | ... | ... | ... 

In [2]:
SELECT *
FROM sales;

SyntaxError: invalid syntax (3323485067.py, line 1)

In [3]:
-- Select the product line, month name, warehouse, and net revenue
SELECT
    product_line,  							 -- Type of product ordered
    TO_CHAR(date, 'Month') AS month, 		 -- Convert 'date' full month name (e.g., 'July')
    warehouse, 								 -- The warehouse that the order was made from
    SUM(total - payment_fee) AS net_revenue  -- Calculate net revenue by subtracting 'payment _fee' from total
FROM sales
WHERE 
    client_type = 'Wholesale'  -- Include only sales to wholesale clients
GROUP BY 
    product_line,           -- Group results by product_line
    warehouse,              -- Group by warehouse to see revenue per location
    TO_CHAR(date, 'Month')  -- Group by the name of the month
ORDER BY 
    product_line ASC,  -- Sort results by product line, alphabetically
    month DESC, 	   -- Then by month name, June to August
    net_revenue DESC;  -- Then by net revenue, descending

SyntaxError: invalid syntax (726851506.py, line 1)

---
# Extended Project below

The finance team is exploring ways to reduce transaction costs and improve profitability. They’ve asked you to determine the most profitable payment method for each warehouse in each month. Calculate the net revenue for each payment method, grouped by warehouse and month, and identify the top payment method for each combination.

In [20]:
-- Rank payment methods by net revenue within each warehouse and month
WITH ranked_payments AS (
	SELECT
		payment,
		EXTRACT(MONTH FROM date) AS month,
		warehouse,
		SUM(total - payment_fee) AS net_revenue,
		ROW_NUMBER() OVER (
			PARTITION BY warehouse, EXTRACT(MONTH FROM date)
			ORDER BY SUM(total - payment_fee) DESC
		) AS rank
	FROM sales
	GROUP BY
		payment,
		EXTRACT(MONTH FROM date),
		warehouse
)

-- Pick the most profitable payment method per warehouse/month
SELECT
    payment,
    month,
    warehouse,
    net_revenue
FROM ranked_payments
WHERE rank = 1
ORDER BY warehouse, month;

,payment,month,warehouse,net_revenue
0,Transfer,6,Central,23453.08
1,Transfer,7,Central,23893.59
2,Transfer,8,Central,31509.00
3,Transfer,6,North,17000.12
4,Transfer,7,North,17585.25
5,Transfer,8,North,23480.13
6,Transfer,6,West,8645.98
7,Transfer,7,West,7606.51
8,Transfer,8,West,6466.42


The marketing team is planning a targeted campaign and wants to know the most popular product lines for retail and wholesale customers.

They have given you the task to find the top 3 most ordered product lines for each client type.

In [30]:
-- Rank product line by number of orders within each client type
WITH ranked_products AS (
	SELECT
		client_type,
    	product_line,
    	COUNT(*) AS count,
		ROW_NUMBER() OVER (
			PARTITION BY client_type
			ORDER BY COUNT(*) DESC
		) AS rank
	FROM sales
	GROUP BY client_type, product_line
)

-- Pick the top 3 most ordered product lines per client type
SELECT
    client_type,
    product_line,
    count
FROM ranked_products
WHERE rank <= 3
ORDER BY
	client_type, 
	rank;

,client_type,product_line,count
0,Retail,Suspension & traction,177
1,Retail,Braking system,175
2,Retail,Electrical system,155
3,Wholesale,Braking system,55
4,Wholesale,Suspension & traction,51
5,Wholesale,Frame & body,38
